In [12]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(
    '/Users/tylerviducic/dev/hockey_analytics/gamescore_model')))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import metrics, preprocessing
import seaborn as sns
from utils import helper_funcs
import glob

In [13]:
even_file_path = '/Users/tylerviducic/dev/hockey_analytics/gamescore_model/data/cleaned_data/even_games/'
even_files = glob.glob(even_file_path + '*.csv')
even_df = pd.concat([pd.read_csv(f) for f in even_files], ignore_index=True)
even_df.drop(columns=['Unnamed: 0'], inplace=True)
even_df.describe()

,icetime,shifts,gameScore,iceTimeRank,I_F_xOnGoal,I_F_xGoals,I_F_xRebounds,I_F_xFreeze,I_F_xPlayStopped,I_F_xPlayContinuedInZone,...,OnIce_A_unblockedShotAttempts,OnIce_A_scoreAdjustedUnblockedShotAttempts,OnIce_A_xGoalsFromxReboundsOfShots,OnIce_A_xGoalsFromActualReboundsOfShots,OnIce_A_reboundxGoals,OnIce_A_xGoals_with_earned_rebounds,OnIce_A_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_A_xGoals_with_earned_rebounds_scoreFlurryAdjusted,home_score,away_score
count,317170.000000,317170.000000,317170.000000,317170.000000,317170.000000,317170.000000,317170.000000,317170.000000,317170.000000,317170.000000,...,317170.000000,317170.000000,317170.000000,317170.000000,317170.000000,317170.000000,317170.000000,317170.000000,317170.000000,317170.000000
mean,990.998490,21.413185,0.452363,5.474843,1.685068,0.156293,0.115754,0.382926,0.055065,0.939114,...,9.071444,9.093792,0.087490,0.093119,0.095601,0.511545,0.513269,0.505697,2.886771,2.617744
std,291.938812,5.429036,0.735345,3.278951,1.396364,0.212206,0.106834,0.323199,0.048267,0.786044,...,4.217612,4.230986,0.045154,0.176545,0.178673,0.289784,0.291800,0.285524,1.680995,1.582254
min,2.000000,1.000000,-2.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,799.000000,18.000000,-0.055000,3.000000,0.693000,0.018000,0.038000,0.146000,0.021000,0.399000,...,6.000000,6.051000,0.055000,0.000000,0.000000,0.298000,0.298000,0.295000,2.000000,1.000000
50%,998.000000,22.000000,0.300000,5.000000,1.435000,0.078000,0.090000,0.323000,0.045000,0.817000,...,9.000000,8.698000,0.082000,0.000000,0.000000,0.472000,0.473000,0.467000,3.000000,2.000000
75%,1189.000000,25.000000,0.850000,8.000000,2.398000,0.209000,0.167000,0.559000,0.079000,1.341000,...,12.000000,11.680000,0.114000,0.140000,0.149000,0.680000,0.683000,0.672000,4.000000,4.000000
max,2334.000000,45.000000,5.650000,13.000000,14.722000,3.598000,1.250000,3.570000,0.693000,8.222000,...,42.000000,41.879000,0.446000,2.259000,2.259000,2.646000,2.746000,2.642000,10.000000,10.000000


In [14]:
odd_file_path = '/Users/tylerviducic/dev/hockey_analytics/gamescore_model/data/cleaned_data/odd_games/'
odd_files = glob.glob(odd_file_path + '*.csv')
odd_df = pd.concat([pd.read_csv(f) for f in odd_files], ignore_index=True)
odd_df.drop(columns=['Unnamed: 0'], inplace=True)
print(odd_df.describe().to_markdown())


|       |    icetime |       shifts |     gameScore |   iceTimeRank |   I_F_xOnGoal |    I_F_xGoals |   I_F_xRebounds |   I_F_xFreeze |   I_F_xPlayStopped |   I_F_xPlayContinuedInZone |   I_F_xPlayContinuedOutsideZone |   I_F_flurryAdjustedxGoals |   I_F_scoreVenueAdjustedxGoals |   I_F_flurryScoreVenueAdjustedxGoals |   I_F_primaryAssists |   I_F_secondaryAssists |   I_F_shotsOnGoal |   I_F_missedShots |   I_F_blockedShotAttempts |   I_F_shotAttempts |    I_F_points |     I_F_goals |   I_F_rebounds |   I_F_reboundGoals |    I_F_freeze |   I_F_playStopped |   I_F_playContinuedInZone |   I_F_playContinuedOutsideZone |   I_F_savedShotsOnGoal |   I_F_savedUnblockedShotAttempts |     penalties |   I_F_penalityMinutes |   I_F_faceOffsWon |     I_F_hits |   I_F_takeaways |   I_F_giveaways |   I_F_lowDangerShots |   I_F_mediumDangerShots |   I_F_highDangerShots |   I_F_lowDangerxGoals |   I_F_mediumDangerxGoals |   I_F_highDangerxGoals |   I_F_lowDangerGoals |   I_F_mediumDangerGoals |   I_F_

In [15]:
even_df['outsample_avg_goal_diff'] = np.where(even_df['season'] == odd_df['season'], odd_df['season_avg_goal_diff'], np.nan)


In [16]:
even_df.dropna(inplace=True)
print(even_df)

       playerId season            name      gameId playerTeam opposingTeam   
0       8477938   2017    Haydn Fleury  2017020023        CAR          MIN  \
1       8477938   2017    Haydn Fleury  2017020068        CAR          WPG   
2       8477938   2017    Haydn Fleury  2017020102        CAR          CGY   
3       8477938   2017    Haydn Fleury  2017020129        CAR          T.B   
4       8477938   2017    Haydn Fleury  2017020152        CAR          STL   
...         ...    ...             ...         ...        ...          ...   
317165  8471388   2014  Patrick Kaleta  2014020798        BUF          OTT   
317166  8471388   2014  Patrick Kaleta  2014021102        BUF          ARI   
317167  8471388   2014  Patrick Kaleta  2014021141        BUF          ARI   
317168  8471388   2014  Patrick Kaleta  2014021162        BUF          CHI   
317169  8471388   2014  Patrick Kaleta  2014021185        BUF          CAR   

       home_or_away  gameDate position  icetime  ...   
0      

In [17]:
rate_stats = ['OnIce_F_xGoals', 'OnIce_F_goals',
              'OnIce_A_xGoals', 'OnIce_A_goals']

for stat in rate_stats:
    even_df[stat] = np.where(True, even_df[stat] / even_df['icetime'] * 3600, 0)

In [18]:
from sklearn import model_selection

features = ['I_F_primaryAssists', 'I_F_secondaryAssists', 'I_F_goals', 'I_F_hits', 'I_F_takeaways', 
            'I_F_dZoneGiveaways', 'shotsBlockedByPlayer','faceoffsWon', 'faceoffsLost', 'penalties', 
            'penaltiesDrawn', 'OnIce_F_xGoals', 'OnIce_F_goals', 'OnIce_A_xGoals', 'OnIce_A_goals']
target = ['outsample_avg_goal_diff']

x_train, x_test, y_train, y_test = model_selection.train_test_split(even_df[features], even_df[target], test_size=0.2)

In [19]:
x_min_max_scaler = preprocessing.MinMaxScaler()
scaled_x_train = x_min_max_scaler.fit_transform(x_train)
scaled_x_test = x_min_max_scaler.fit_transform(x_test)

y_min_max_scaler = preprocessing.MinMaxScaler()
scaled_y_train = y_min_max_scaler.fit_transform(y_train)
scaled_y_test = y_min_max_scaler.fit_transform(y_test)



In [20]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()

In [21]:
model.fit(scaled_x_train, scaled_y_train)

LinearRegression()

In [22]:
print(f'coeficcients: ')
for feature, coef in zip(features, model.coef_[0]):
    print(f'{feature}: {coef}')

coeficcients: 
I_F_primaryAssists: -0.0006149180964416927
I_F_secondaryAssists: 0.00032572918353260434
I_F_goals: 0.0001417706232405404
I_F_hits: 0.0008662894283934977
I_F_takeaways: 0.0010747107674628402
I_F_dZoneGiveaways: 0.0013243926522215378
shotsBlockedByPlayer: -0.001688309562413088
faceoffsWon: -0.001958538373428245
faceoffsLost: 0.0025655382886727402
penalties: -0.0004175094904184307
penaltiesDrawn: -0.0024967028375462545
OnIce_F_xGoals: 0.01067776580667419
OnIce_F_goals: 0.008753136297430892
OnIce_A_xGoals: 0.02235414119209815
OnIce_A_goals: 0.0014780738238758108
